In [1]:
# Sqlalchemy functions
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *


#langchain functions
from langchain import hub
from langchain.agents import AgentExecutor, tool, load_tools
from crewai import Agent, Task, Crew, Process
from langchain.llms import VertexAI
from vertexai.language_models import TextGenerationModel

#other necessary functions
import os
import json
import pandas as pd
pd.set_option('display.max_colwidth', None)  # Set maximum width to terminal width
pd.set_option('display.expand_frame_repr', True)  # Expand DataFrame representation


In [2]:
# Example usage:
llm = VertexAI(
    model_name='gemini-pro',
    temperature=0,
    top_k=1,
)
tools_name=["llm-math"]
tools=load_tools(tools_name,llm)

/home/user/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


In [3]:
# Define your agents with roles and goals
coder = Agent(
    role='Extractor Specialist',
    goal="""As an expert in extracting product quantity details, your objective is to analyze product titles provided by the user and produce a JSON format with the following fields:
      - "net_content_quantity"
      - "net_content_uom,"
      - "count_per_pack,"
      - "multi_quantity,"
      - "total_quantity."

       Follow the guidelines to set count per pack to 1 if not given and multi quantity is, and vice versa.

       Guidelines:
        - If count per pack is not given and multi quantity is given, set count per pack to 1.
        - If multi quantity is not given and count per pack is given, set multi quantity to 1.

       For instance, given the title "V8 +Energy, Juice Drink with Green Tea, Peach Mango, 8 oz. Can (4 packs of 6, Total of 24)," your response should be:
         {"net_content_quantity":8 ,"net_content_uom": "oz", "count_per_pack": 6, "multi_quantity": 4, "total_quantity": 24}

       Few More examples:
        - Title: V8, Strawberry Banana, 8 Ounce, 6 Count (Pack of 4)
          Response: {"net_content_quantity":8 ,"net_content_uom": "Ounce", "count_per_pack": 4, "multi_quantity": 6, "total_quantity": 24}
        - Title: V8 100% Vegetable Juice, Spicy Hot, 5.5 oz. Can, 6 Count
          Response: {"net_content_quantity":5.5 ,"net_content_uom": "oz", "count_per_pack": 1, "multi_quantity": 6, "total_quantity": 6}
        - Title: Orbit Peppermint Sugarfree Gum, 3 Pack
          Response: {"net_content_quantity":null ,"net_content_uom": null, "count_per_pack": 1, "multi_quantity": 3, "total_quantity": 3}
        - Title: EO Botanical Sulfate Free Conditioner with Keratin, Coconut and Hibiscus, 8.4 oz
          Response: {"net_content_quantity":8.4 ,"net_content_uom": "oz", "count_per_pack": 1, "multi_quantity": 1, "total_quantity": 1}
       """,
    backstory="""As an experienced data analyst, your role is pivotal in extracting accurate product information. Your focus is on creating a structured JSON response from the provided product titles.""",
    verbose=True,
    allow_delegation=False,
    llm=llm,  # was defined above in the file
)

manager = Agent(
    role='Validation Supervisor',
    goal="""Your task is to validate the correctness of the JSON output generated by the Extractor Specialist (Coder). Multiply the "count_per_pack" value by the "multi_quantity" value and ensure it matches the "total_quantity" provided in the JSON. If incorrect, ask the Coder to revise.""",
    backstory="""As a manager, your responsibility is to review and verify the accuracy of the JSON data produced by the Extractor Specialist. Performing calculations on the provided data ensures the integrity of the information.""",
    verbose=True,
    allow_delegation=True,
    llm=llm,
    tools=tools
)



In [4]:
agents =[coder,manager]
task_descriptions = [
    "Extract pertinent information in JSON format from the given product title: {product_title}",
    "Review and validate the JSON output provided by the Extractor Specialist (Coder) for the product title: {product_title} If the extracted JSON is valid, return the JSON data without additional wording. If the JSON is not valid, correct the values using the llm-math tool. Ensure the corrected JSON includes the fields 'net_content_quantity','net_content_uom', 'count_per_pack', 'multi_quantity', and 'total_quantity'. The corrected JSON should be devoid of extra leading and trailing wordings. If the calculated total quantity matches the provided value, return the corrected JSON as the final answer. If the JSON is still not valid after correction, communicate the issue to the Coder for further adjustments."
]


In [5]:
# Sqlalchemy functions
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *


#langchain functions
from langchain import hub
from langchain.agents import AgentExecutor, tool, load_tools
from crewai import Agent, Task, Crew, Process
from langchain.llms import VertexAI
from vertexai.language_models import TextGenerationModel

#other necessary functions
import os
import json
import pandas as pd



class ProductInfoExtractor:
    def __init__(self,engine):
        self.engine = engine
        self.output_directory = None
        self.target_attribute = None


    def load_data(self, schema_name, table_name, uids, target_attribute):
        """
        Retrieve rows where the specified column is NOT NULL.

        Parameters:
            schema_name (str): Name of the database schema.
            table_name (str): The name of the table.
            uids (list): List of column names used as unique identifiers.
            target_attribute (str): The column to be retrieved.

        Returns:
            pd.DataFrame: DataFrame containing training data.
        """
        self.target_attribute = target_attribute

        if self.engine:
            try:
                query = f"SELECT {','.join(uids)},{self.target_attribute} FROM {schema_name}.{table_name} WHERE {self.target_attribute} is not null and {self.target_attribute}!=''"
                data = pd.read_sql_query(query, self.engine)
                return data
            except Exception as e:
                raise RuntimeError(f"Failed with error: {str(e)}")
        else:
            raise RuntimeError('Connection Issue: Engine is not available.')


    def execute_tasks(self,agents, task_descriptions, product_title):
        """
            Creates and executes tasks with a crew of agents using zip.

            Args:
                agents (list): List of Agent objects to assign tasks to.
                task_descriptions (list): List of task descriptions,
                    each containing product_title placeholders for formatting.
                product_title (str): The specific product title for the tasks.

            Returns:
                The result of executing the tasks by the crew.
        """

        tasks = [Task(description=desc.format(product_title=product_title), agent=agent) for desc, agent in zip(task_descriptions, agents)]
        tech_crew = Crew(agents=agents, tasks=tasks, process=Process.sequential)
        return tech_crew.kickoff()
    
    
    def extract_attributes(self,data_frame,agents, task_descriptions):
        """
        Extracts attributes from a DataFrame by executing tasks with assigned agents.

        Args:
            data_frame (pd.DataFrame): DataFrame containing product titles.
            agents (list): List of Agent objects to execute the tasks.
            task_descriptions (list): List of task descriptions for each product title.

        Returns:
            The DataFrame with an added 'Attribute_extraction' column containing the extracted attributes.
        """
        
        data_frame['Attribute_extraction']=data_frame[self.target_attribute].apply(lambda x: self.execute_tasks(agents, task_descriptions,x))
        return data_frame


In [6]:
# Establish a connection to the BigQuery database
engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')

/tmp/ipykernel_11805/2832908811.py:2: SADeprecationWarning: The dbapi() classmethod on dialect classes has been renamed to import_dbapi().  Implement an import_dbapi() classmethod directly on class <class 'pybigquery.sqlalchemy_bigquery.BigQueryDialect'> to remove this warning; the old .dbapi() classmethod may be maintained for backwards compatibility.
  engine = create_engine('bigquery://us-gcp-ame-con-52dbb-sbx-1')


In [7]:
extractor = ProductInfoExtractor(engine)

In [8]:
# Calling `load_data` function with specific parameters:
df = extractor.load_data(schema_name='data_quality',
                   table_name='meta_Prime_Pantry',
                   uids=['asin'], 
                   target_attribute='title'
                   )
df.shape

/home/user/.local/lib/python3.10/site-packages/google/cloud/bigquery/client.py:572: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


(10813, 2)

In [9]:
#,'B015IOAG7O','B015IOABMY'
df= df[df.asin.isin(['B00WS3YLQ6',
'B01BODU45C',
'B01GCT2CJY',
'B00TDFRHN6',
'B010Q32SLY',
'B00OTH6HCA',
'B014M6LTSO',
'B010SI8YJW',
'B010NBPJGU',
'B00MY2T3K0',
'B00QJGTYWO',
'B00I9KRBRS',
'B00MTSHIBK',
'B00T7ATYWO',
'B010UO14R8',
'B00SMYT6PG',
'B014F3OYCC',
'B01BTOD3Z4',
'B00NQ2PDFG',
'B019H0L6C4',
'B00V1O5X3I'
])]

In [10]:
result_df = extractor.extract_attributes(df,agents,task_descriptions)
result_df



> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: {"net_content_quantity":8 ,"net_content_uom": "Ounce", "count_per_pack": 6, "multi_quantity": 4, "total_quantity": 24}

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 6 * 4Answer: 24Do I need to use a tool? No
Final Answer: {"net_content_quantity":8 ,"net_content_uom": "Ounce", "count_per_pack": 6, "multi_quantity": 4, "total_quantity": 24}

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: {"net_content_quantity":1.5 ,"net_content_uom": "Ounce", "count_per_pack": 1, "multi_quantity": 1, "total_quantity": 1}

> Finished chain.


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 1 * 1Answer: 1Do I need to use a tool? No
Final Answer: {"net_content_quantity":1.5 ,"net_content_uom": "Oun

,asin,title,Attribute_extraction
29,B00WS3YLQ6,"V8, Strawberry Banana, 8 Ounce, 6 Count (Pack of 4)","{""net_content_quantity"":8 ,""net_content_uom"": ""Ounce"", ""count_per_pack"": 6, ""multi_quantity"": 4, ""total_quantity"": 24}"
40,B01BODU45C,"A+D Original Diaper Rash Ointment, Baby Skin Protectant With Lanolin and Petrolatum, Seals Out Wetness, Helps Prevent Diaper Rash, 1.5 Ounce Tube","{""net_content_quantity"":1.5 ,""net_content_uom"": ""Ounce"", ""count_per_pack"": 1, ""multi_quantity"": 1, ""total_quantity"": 1}"
67,B00TDFRHN6,"BSN SYNTHA-6 Protein Powder, Strawberry Milkshake, 5.0 Pound","{""net_content_quantity"":5.0 ,""net_content_uom"": ""Pound"", ""count_per_pack"": 1, ""multi_quantity"": 1, ""total_quantity"": 1}"
78,B010Q32SLY,"C2O Pure Coconut Water, With Pulp, 17.5 Ounce (Pack of 12)","{""net_content_quantity"":17.5 ,""net_content_uom"": ""Ounce"", ""count_per_pack"": 12, ""multi_quantity"": 1, ""total_quantity"": 12}"
82,B00OTH6HCA,"EAS Myoplex Original Ready-to-Drink Nutrition Shake, Chocolate Ice Cream, 16.9 fl.oz,4 Count","{""net_content_quantity"":16.9 ,""net_content_uom"": ""fl.oz"", ""count_per_pack"": 4, ""multi_quantity"": 1, ""total_quantity"": 4}"
99,B014M6LTSO,"Jif Creamy Peanut Butter Twin Pack, 80 Ounce","{""net_content_quantity"":80 ,""net_content_uom"": ""Ounce"", ""count_per_pack"": 2, ""multi_quantity"": 1, ""total_quantity"": 160}"
1244,B010SI8YJW,"Os-Cal 500 D Calcium D3 200 I.U Coated Caplets, 210 Count","{""net_content_quantity"":null,""net_content_uom"":null,""count_per_pack"":210,""multi_quantity"":1,""total_quantity"":210}"
1268,B01GCT2CJY,"Diet Pepsi, Bottles 16.9 Fl Oz, 6 Count","{""net_content_quantity"":16.9 ,""net_content_uom"": ""Fl Oz"", ""count_per_pack"": 6, ""multi_quantity"": 1, ""total_quantity"": 6}"
1908,B010NBPJGU,"Larabar Gluten Free Bars, Coconut Cream Pie, 1.7 oz Bars (16 Count)","{""net_content_quantity"":1.7 ,""net_content_uom"": ""oz"", ""count_per_pack"": 16, ""multi_quantity"": 1, ""total_quantity"": 16}"
2175,B00V1O5X3I,"Near East Whole Grain Blends, Roasted Pecan &amp; Garlic, with Brown Rice and Bulgur, 5.4oz Box","{""net_content_quantity"":5.4 ,""net_content_uom"": ""oz"", ""count_per_pack"": 1, ""multi_quantity"": 1, ""total_quantity"": 1}"
